In [5]:
import os
import replicate
from langchain.prompts import ChatPromptTemplate
import argparse
from get_embedding_function import get_embedding_function
from langchain.vectorstores.chroma import Chroma

from langchain_community.llms.ollama import Ollama

from dotenv import load_dotenv
import os

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')

DATA_PATH = "data"

CHROMA_PATH = "chroma"
PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""

# specially fine-tuned for our task
# Note: need `REPLICATE_API_TOKEN` as an environment variable
def query_finetuned_model(query_text: str):
    
    training = replicate.trainings.get("h6ck7vye45rgm0cf29br7wx2pr")

    embedding_function = get_embedding_function()
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

    # Search the DB.
    results = db.similarity_search_with_score(query_text, k=5)
    sources = [doc.metadata.get("id", None) for doc, _score in results]

    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)

    response_text = replicate.run(
      training.output["version"],
      input={"prompt": prompt, "stop_sequences": "</s>"}
    )
    for s in response_text:
      print(s, end="", flush=True)

    print(f"Sources: {sources}")


# base llama3 model with 70B parameters
# Note: need `REPLICATE_API_TOKEN` as an environment variable
def query_base_70B_model(query_text: str):
    
    embedding_function = get_embedding_function()
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

    # Search the DB.
    results = db.similarity_search_with_score(query_text, k=5)
    sources = [doc.metadata.get("id", None) for doc, _score in results]

    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)

    # The meta/llama-2-70b-chat model can stream output as it's running.
    for event in replicate.stream(
        "meta/llama-2-70b-chat",
        input={
            "top_k": 0,
            "top_p": 1,
            "prompt": prompt,
            "temperature": 0.5,
            "system_prompt": "You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.",
            "length_penalty": 1,
            "max_new_tokens": 500,
            "min_new_tokens": -1,
            "prompt_template": "<s>[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n{prompt} [/INST]",
            "presence_penalty": 0
        },
    ):
        print(str(event), end="")
      
    print(f"\nSources: {sources}")



def query_local_rag(query_text: str):
    # Prepare the DB.
    embedding_function = get_embedding_function()
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

    # Search the DB.
    results = db.similarity_search_with_score(query_text, k=5)

    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)
    #print(prompt)

    # https://ollama.com/library/llama3:8b
    model = Ollama(model="llama3:8b")
    response_text = model.invoke(prompt)

    sources = [doc.metadata.get("id", None) for doc, _score in results]
    formatted_response = f"Response: {response_text}\nSources: {sources}"
    print(formatted_response)
    return response_text


In [3]:
query_local_rag('What phenomenon scares the bejesus out of Chandler?')

Response: According to the context, the phenomenon that scares the "bejesus" out of Chandler is Michael Flatley's dance performance in "Lord of the Dance". Specifically, it's his legs flailing about as if independent from his body!
Sources: ['QA.txt:N/A:6', 'QA.txt:N/A:20', 'QA.txt:N/A:46', 'QA.txt:N/A:63', 'QA.txt:N/A:192']


'According to the context, the phenomenon that scares the "bejesus" out of Chandler is Michael Flatley\'s dance performance in "Lord of the Dance". Specifically, it\'s his legs flailing about as if independent from his body!'

In [20]:
from langchain_community.document_loaders import JSONLoader
from pathlib import Path
from pprint import pprint

for filename in os.listdir(DATA_PATH):
    file_path = os.path.join(DATA_PATH, filename)

    if file_path.endswith('jsonl'):

        pprint(Path(file_path).read_text())

        

        loader = JSONLoader(
            file_path=file_path,
            
        )
        

             

('{"prompt": "In Season 1, how is Chandler Bing\'s personality initially '
 'described?", "completion": "Chandler is introduced as a sarcastic person '
 'with a heightened fear of commitment."}\n'
 '{"prompt": "In Season 1, what comedic dynamic does Chandler share with his '
 'roommate Joey?", "completion": "Chandler constantly makes fun of Joey, '
 'highlighting their playful and sarcastic dynamic."}\n'
 '{"prompt": "In Season 1, what recurring problem does Chandler face with '
 'Janice?", "completion": "Chandler finds himself repeatedly stumbling across '
 'Janice, an annoying woman whom he has to break up with three times within a '
 'year."}\n'
 '{"prompt": "In Season 1, what triggers Chandlerâ€™s smoking relapse?", '
 '"completion": "Chandler\'s addiction to cigarettes relapses while he is '
 'helping Joey practice for an audition for the role of a person who has to '
 'smoke."}\n'
 '{"prompt": "In Season 1, why does Chandler experience sadness after Monica '
 'breaks up with Alan

In [9]:
%ls

 Volume in drive C has no label.
 Volume Serial Number is 3518-43E0

 Directory of c:\Users\gcmn1\OneDrive\Desktop\Llama-Langchain-RAG

04/28/2024  01:55 AM    <DIR>          .
04/28/2024  12:07 AM    <DIR>          ..
04/27/2024  01:46 PM             6,148 .DS_Store
04/28/2024  01:36 AM               137 .env
04/27/2024  01:46 PM                43 .gitattributes
04/27/2024  01:46 PM                 4 .gitignore
04/28/2024  12:05 AM    <DIR>          __pycache__
04/28/2024  12:05 AM    <DIR>          chroma
04/28/2024  02:07 AM    <DIR>          data
04/28/2024  12:05 AM               581 get_embedding_function.py
04/28/2024  02:10 AM             7,766 main.ipynb
04/27/2024  08:43 PM             3,634 populate_database.py
04/28/2024  01:36 AM             4,487 query_data.py
04/28/2024  01:55 AM             3,312 README.md
04/28/2024  01:57 AM               373 requirements.txt
              10 File(s)         26,485 bytes
               5 Dir(s)  294,996,422,656 bytes free
